# Visualização dos Resultados

## Setup do Notebook

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [2]:
layout_settings = {
    "width": 720, 
    "height": 480,
    "font": dict(size=16),
    "margin": dict(l=10, r=10, t=40, b=10),
}

all_colors = px.colors.qualitative.Plotly

In [3]:
def get_rgba(hex, opacity=0.3):
    return "rgba(%d, %d, %d, %.2f)" % (tuple(int(hex.lstrip('#')[i:i+2], 16) for i in (0, 2, 4)) + (opacity,))

def add_mean_std_trace(fig, df, name, idx, y_col, x_col="n"):
    df_mean = df.groupby(df.index).mean()
    df_std = df.groupby(df.index).std()

    color = all_colors[idx % len(all_colors)]
    fig.add_trace(go.Scatter(
        name=name,
        x=df_mean[x_col],
        y=df_mean[y_col],
        mode="markers", # lines+markers
        line=dict(color=color),
    ))
    # fig.add_traces(go.Scatter(
    #     name=name + " upper",
    #     x=df_mean[x_col],
    #     y=df_mean[y_col] + df_std[y_col],
    #     mode="lines",
    #     line=dict(width=0),
    #     showlegend=False
    # ))
    # fig.add_traces(go.Scatter(
    #     name=name + " lower",
    #     x=df_mean[x_col],
    #     y=df_mean[y_col] - df_std[y_col],
    #     line=dict(width=0),
    #     mode='lines',
    #     fillcolor=get_rgba(color),
    #     fill='tonexty',
    #     showlegend=False,
    # ))

## Experimentos

In [4]:
runs = [
    (
        "Sondagem Linear",
        [
            # pd.read_csv("outputs/run_inserts_linear.tsv", sep="\t")
            # pd.read_csv("outputs/run_in_out_linear.tsv", sep="\t")
            # pd.read_csv("outputs/run_in_linear.tsv", sep="\t")
            pd.read_csv("outputs/run_in_out_net_linear.tsv", sep="\t")
        ],
    ),
    (
        "Sondagem Quadratica",
        [
            # pd.read_csv("outputs/run_inserts_quadratic.tsv", sep="\t")
            # pd.read_csv("outputs/run_in_out_quad.tsv", sep="\t")
            # pd.read_csv("outputs/run_in_quad.tsv", sep="\t")
            pd.read_csv("outputs/run_in_out_net_quad.tsv", sep="\t")
        ],
    ),
]
runs


[('Sondagem Linear',
  [         n    ops  comparisons  load_factor  inserts  removes  finds  \
   0       10     10           10     0.001273       10        0      0   
   1       20     20           20     0.002547       20        0      0   
   2       30     30           30     0.003820       30        0      0   
   3       40     40           40     0.005094       40        0      0   
   4       50     50           51     0.006367       50        0      0   
   ...    ...    ...          ...          ...      ...      ...    ...   
   1595  5040  14960        27957     0.641793    10000     4960      0   
   1596  5030  14970        28010     0.640520    10000     4970      0   
   1597  5020  14980        28052     0.639246    10000     4980      0   
   1598  5010  14990        28076     0.637973    10000     4990      0   
   1599  5000  15000        28110     0.636699    10000     5000      0   
   
         groups_total  groups_mean  groups_stdev  groups_min  groups_max  


In [5]:
fig = go.Figure()
for idx, (name, dfs) in enumerate(runs):
    df = pd.concat(dfs)
    add_mean_std_trace(fig, df, name, idx, "comparisons", "ops")

fig.update_layout(title="Comparações", xaxis_title='Ops', yaxis_title="Comparações", **layout_settings)
fig.show()

In [6]:
fig = go.Figure()
for idx, (name, dfs) in enumerate(runs):
    df = pd.concat(dfs)
    add_mean_std_trace(fig, df, name, idx, "groups_mean", "ops")

fig.update_layout(title="Tamanho médio de grupos por número de operações", xaxis_title='Ops', yaxis_title="Tamanho do Grupo", **layout_settings)
fig.show()

In [ ]:
fig = go.Figure()
for idx, (name, dfs) in enumerate(runs):
    df = pd.concat(dfs)
    add_mean_std_trace(fig, df, name, idx, "comparisons", "ops")

fig.update_layout(title="Comparações", xaxis_title='Ops', yaxis_title="Comparações", **layout_settings)
fig.show()